In [ ]:
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)
from dotenv import load_dotenv
import os

In [ ]:
class CAMELAgent:
    def __init__(
        self,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.init_messages()

    def reset(self) -> None:
        self.init_messages()
        return self.stored_messages

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)
        return self.stored_messages

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        messages = self.update_messages(input_message)

        output_message = self.model(messages)
        self.update_messages(output_message)

        return output_message

In [ ]:


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

assistant_role_name = "Joe Roegan"
user_role_name = "Sam Altman"

In [ ]:
assistant_inception_prompt = """Never forget you're {assistant_role_name}. Never flip roles! 


This is the situation :
  - I'm {user_role_name} and you're interviewing me for your podcast
  - this interview is very important for you, you don't have to mess things up.
  - there is a director called Director who will give you some instructions sometimes.

This is the way you have to behave : 
  - say 'damn' sometimes when you speak
  - Never call {user_role_name} by his name ! this is really important ! 

Do not change roles!
Do not speak from the perspective of {user_role_name} or from the perspective of the director
Stop speaking the moment you finish speaking from your perspective.
"""

user_inception_prompt = """Nerver forget you're {user_role_name}, the famous artist. Never flip roles! 

This is the way you have to behave : 
  - You say 'man' a lot of times when you speaking to someone.

This is the situation : 
  - I'm {assistant_role_name} and I'm interviewing you for my podcast.
  - you're here because you're manager wanted it. You don't like me, but you cannot say it to me.
  - instead of being agressive, you'll mispell my name by calling me 'Jay' sometimes instead of 'Joe'. If I corrects you, you will say sorry, but continue to mispell my name. 

Do not change roles!
Do not speak from the perspective of {assistant_role_name} or from the perspective of the director
Stop speaking the moment you finish speaking from your perspective.
"""

In [ ]:
def get_sys_msgs(assistant_role_name: str, user_role_name: str):
    assistant_sys_template = SystemMessagePromptTemplate.from_template(
        template=assistant_inception_prompt
    )
    assistant_sys_msg = assistant_sys_template.format_messages(
        assistant_role_name=assistant_role_name,
        user_role_name=user_role_name,
    )[0]

    user_sys_template = SystemMessagePromptTemplate.from_template(
        template=user_inception_prompt
    )
    user_sys_msg = user_sys_template.format_messages(
        assistant_role_name=assistant_role_name,
        user_role_name=user_role_name,
    )[0]

    return assistant_sys_msg, user_sys_msg

In [ ]:
assistant_sys_msg, user_sys_msg = get_sys_msgs(
    assistant_role_name, user_role_name
)
assistant_agent = CAMELAgent(assistant_sys_msg, ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.2))
user_agent = CAMELAgent(user_sys_msg, ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0.2))

In [ ]:


# Reset agents
assistant_agent.reset()
user_agent.reset()



In [ ]:
chat_turn_limit, n = 30, 0
conversation=[]


In [ ]:
director_msg = HumanMessage(
    content=f"Director instruction : start with a warm welcome !")
assistant_msg = assistant_agent.step(director_msg)

transcript = f"{assistant_role_name}:\n\n{assistant_msg.content}\n\n"
print(transcript)
conversation.append(transcript)

In [ ]:

# n += 1
user_ai_msg = user_agent.step(assistant_msg)
user_msg = HumanMessage(content=user_ai_msg.content)
transcript = f"{user_role_name}:\n\n{user_msg.content}\n\n"
print(transcript)
conversation.append(transcript)


assistant_ai_msg = assistant_agent.step(user_msg)
assistant_msg = HumanMessage(content=assistant_ai_msg.content)
transcript = f"{assistant_role_name}:\n\n{assistant_msg.content}\n\n"
print(transcript)
conversation.append(transcript)

In [ ]:
director_msg = HumanMessage(
    content=f"Director instruction : you're talking too much ! be shorter")
assistant_msg = assistant_agent.step(director_msg)

transcript = f"{assistant_role_name}:\n\n{assistant_msg.content}\n\n"
print(transcript)
conversation.append(transcript)

In [ ]:
assistant_agent.stored_messages